In [9]:
import time
import mysql.connector
import mysql.connector
import json
import pandas as pd
import datetime
import re

In [2]:
def connect_to_mysql(database):
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="123456",
        database=database
    )
    return conn

# 从MySQL中检索数据
def fetch_data_from_mysql(conn, table_name, *columns):
    columns_str = ', '.join(columns)
    query = "SELECT {} FROM `{}`".format(columns_str, table_name)
    df = pd.read_sql(query, conn)
    return df

In [3]:
def load_json(comment):
    if pd.isna(comment) or comment.strip() == '':
        return None  # 返回 None 如果字符串为空或为 NaN
    try:
        return json.loads(comment)  # 尝试解析 JSON
    except json.JSONDecodeError:
        return None  # 如果不是有效的 JSON，返回 None

In [4]:
#输入一段字符串 输出这段字符串包含的文件是否涉及多语言
def dulge_ifMpl(a):
    str1 = ''.join(str(i) for i in a)
    flag = 0
    c = 0
    py = 0
    lua1 = 0
    java = 0
    ruby = 0
    cuda = 0
    js = 0
    cp1 = 0
    cx = 0
    go1 = 0
    ts = 0
    swift = 0
    Haskell1 = 0
    erl = 0
    kt = 0
    lsp = 0
    rs = 0
    vue = 0
    groovy = 0
    scala = 0
    CoffeeScript = 0
    perl = 0
    Clojure = 0
    F1 = 0
    PHP = 0
    cp2 = 0
    expect = 0
    sh = 0
    glsl = 0
    j2 = 0
    if any(ext in str1 for ext in [".cpp'", ".h'", ".c'", ".hpp'", ".cc'", ".hxx'", ".cxx'", ".hh'"]):
        c = 1
    if any(ext in str1 for ext in [".cuh'", ".cu'"]):
        cuda = 1
    if (".py'" in str1):
        py = 1
    if (".lua'" in str1):
        lua1 = 1
    if any(ext in str1 for ext in [".java'", ".jav'"]):
        java = 1
    if (".rb'" in str1):
        ruby = 1
    if (".js'" in str1):
        js = 1
    if (".m'" in str1):
        cp1 = 1  # Objective-C
    if (".mm'" in str1):
        cp2 = 1  # Objective-C++
    if (".cs'" in str1):
        cx = 1  # C#
    if (".go'" in str1):
        go1 = 1
    if (".ts'" in str1):
        ts = 1  # TypeScript
    if (".swift'" in str1):
        swift = 1
    if (".hs'" in str1):
        Haskell1 = 1
    if (".erl'" in str1):
        erl = 1  # Erlang
    if (".kt'" in str1):
        kt = 1  # Kotlin
    if (".lsp'" in str1):
        lsp = 1  # LISP
    if (".rs'" in str1):
        rs = 1  # Rust
    if (".vue'" in str1):
        vue = 1  # vue
    if (".groovy'" in str1):
        groovy = 1  # groovy
    if any(ext in str1 for ext in [".sc'", ".scala'"]):
        scala = 1  # Scala
    if any(ext in str1 for ext in [".coffee'", ".litcoffee'"]):
        CoffeeScript = 1  # CoffeeScript
    if any(ext in str1 for ext in [".pl'", ".prl'", ".pm'", ".perl'"]):
        perl = 1  # perl
    if any(ext in str1 for ext in [".clj'", ".cljs'", ".cljc'", ".edn'"]):
        Clojure = 1  # Clojure
    if any(ext in str1 for ext in [".fs'", ".fsi'", ".fsx'", ".fsscript'"]):
        F1 = 1  # F#
    if any(ext in str1 for ext in [".php'", ".php3'", ".php4'", ".php5'", ".php7'"]):
        PHP = 1  # PHP
    if (".expect'" in str1):
        expect = 1
    if (".sh'" in str1):
        sh = 1
    if (".glsl'" in str1):
        glsl = 1
    if (".j2'" in str1):
        j2 = 1
    if ((c + py + lua1 + java + ruby + cuda + js + cp1 + cp2 + cx + go1 + ts + swift + Haskell1 + erl + kt + lsp + rs + vue + groovy + scala + CoffeeScript + perl + Clojure + F1 + PHP + expect + sh + glsl + j2) >= 2):
        flag = 1
    return flag

In [5]:
def convert_issueData_to_sentimentAnalysis(df_issue_content, df_pr_data, df_issue_data, list_sentiAnalysis):
    """
    将issue的comments转化为情感分析DataFrame。

    参数:
    df_issue_content: 包含IssueContent的DataFrame。
    df_pr_data: 包含Issue是否为多语言issue的DataFrame。
    df_issue_data: 包含Issue类别的DataFrame。
    list_sentiAnalysis: 用于存储新行数据的列表。

    返回:
    None。
    """
    # 遍历 df_issue_content 并将数据添加到 new_rows 列表
    for index, row in df_issue_content.iterrows():
        relatedPrID = df_issue_data.loc[df_issue_data['IssueID'] == row['IssueID'], 'RelatedPrId'].values[0]
        issueCategory = df_issue_data.loc[df_issue_data['IssueID'] == row['IssueID'], 'Category'].values[0]
        isMPLF = df_pr_data.loc[df_pr_data['PrRelatedIssueId'] == row['IssueID'], 'IsMPLF'].values[0]
#         new_row = {
#             'TextContent': row['IssueContent'],
#             'TextType': 'IssueBody',
#             'url': row['url'],
#             'Creator': row['IssueCreator'],
#             'CreatedAt': row['IssueCreatedAt'],
#             'IssueID': row['IssueID'],
#             'RelatedPrID': relatedPrID,
#             'IssueCategory': issueCategory,
#             'IsMPLF': isMPLF
#         }
#         # 将新行字典添加到新行列表
#         list_sentiAnalysis.append(new_row)
        
         # 处理 files 列，当其不为空
        if row['IssueComments']:
            comments = row['IssueComments']
            for comment in comments:
                comment_row = {
                    'TextContent': comment['body'],
                    'TextType': 'IssueComment',
                    'url': comment['url'],
                    'Creator': comment['creator'],
                    'CreatedAt': datetime.datetime.strptime(comment['createdAt'], '%Y-%m-%dT%H:%M:%SZ'),
                    'IssueID': row['IssueID'],
                    'RelatedPrID': relatedPrID,
                    'IssueCategory': issueCategory,
                    'IsMPLF': isMPLF
                }
                list_sentiAnalysis.append(comment_row)

In [6]:
def convert_prData_to_sentimentAnalysis(df_pr_content, df_pr_data, df_issue_data, list_sentiAnalysis):
    """
    将issue的content和comments转化为情感分析DataFrame。

    参数:
    df_issue_content: 包含IssueContent和Comments的DataFrame。
    df_pr_data: 包含Issue是否为多语言issue的DataFrame
    df_issue_data: 包含Issue类别的DataFrame
    list_sentiAnalysis: 用于存储新行数据的列表。

    返回:
    None: 直接修改提供的新行列表。
    """
    # 遍历 df_issue_content 并将数据添加到 new_rows 列表
    for index, row in df_pr_content.iterrows():
        issueID = df_pr_data.loc[df_pr_data['PrID'] == row['PrID'], 'PrRelatedIssueId'].values[0]
        # 判断该pull request对应的issue在不在已分类的issues中，不在则过滤掉
        if(issueID not in df_issue_data['IssueID'].values):
            continue
        
        issueCategory = df_issue_data.loc[df_issue_data['IssueID'] == issueID, 'Category'].values[0]
        isMPLF = df_pr_data.loc[df_pr_data['PrRelatedIssueId'] == issueID, 'IsMPLF'].values[0]
#         new_row = {
#             'TextContent': row['PrContent'],
#             'TextType': 'PrBody',
#             'url': row['url'],
#             'Creator': row['PrCreator'],
#             'CreatedAt': row['PrCreatedAt'],
#             'IssueID': issueID,
#             'RelatedPrID': row['PrID'],
#             'IssueCategory': issueCategory,
#             'IsMPLF': isMPLF
#         }
#         # 将新行字典添加到新行列表
#         list_sentiAnalysis.append(new_row)
        
         # 处理 files 列，当其不为空
        if row['PrComments']:
            comments = row['PrComments']
            for comment in comments:
                comment_row = {
                    'TextContent': comment['body'],
                    'TextType': 'PrComment',
                    'url': comment['url'],
                    'Creator': comment['creator'],
                    'CreatedAt': datetime.datetime.strptime(comment['createdAt'], '%Y-%m-%dT%H:%M:%SZ'),
                    'IssueID': issueID,
                    'RelatedPrID': row['PrID'],
                    'IssueCategory': issueCategory,
                    'IsMPLF': isMPLF
                }
                list_sentiAnalysis.append(comment_row)

In [7]:
def preprocess_text(text):
    # 替换 URL、filePath、电子邮件、代码片段等为特定标记
    def replace_patterns(text):
        url_pattern = re.compile(r'(?:https?://|www\.)(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        email_pattern = re.compile(r'\b[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+\b')
        code_pattern = re.compile(r'`[^`]+`')  # Markdown inline code
        
        text = url_pattern.sub("[url]", text)
        text = email_pattern.sub("[email]", text)
        text = code_pattern.sub("[code]", text)

        return text

    # 从文本中识别并替换标记
    text = replace_patterns(text)
    
    # 去除特殊字符和非字母数字表情字符
    pattern = r'[^a-zA-Z0-9\s.,!?;:\[\] \U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FAFF]'
    cleaned_text = re.sub(pattern, '', text)
    
    # 去除多余空格：替换多个空格为一个空格，并去掉文本开头和结尾的空格
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [11]:
conn = connect_to_mysql('deep_learning')
df_incubator_mxnet_issue_data = fetch_data_from_mysql(conn, 'incubator-mxnet_issue_data', 'IssueID', 'RelatedPrId', 'Category')
df_incubator_mxnet_pr_data = fetch_data_from_mysql(conn, 'incubator-mxnet_pr_data', 'PrID', 'PrRelatedIssueId', 'Files')
df_issue_content_mx = fetch_data_from_mysql(conn, 'issue_content_mx', 'IssueID', 'IssueContent', 'url', 'IssueCreator', 'IssueCreatedAt', 'IssueCommentsNum', 'IssueComments')
df_pr_content_mx = fetch_data_from_mysql(conn, 'pr_content_mx', 'PrID', 'PrContent', 'url', 'PrCreator', 'PrCreatedAt', 'PrCommentsNum', 'PrComments')
conn.close()

df_incubator_mxnet_issue_data.drop_duplicates(inplace=True)
df_incubator_mxnet_pr_data.drop_duplicates(inplace=True)
df_issue_content_mx.drop_duplicates(inplace=True)
df_pr_content_mx.drop_duplicates(inplace=True)

print('数据条数：', df_incubator_mxnet_issue_data.shape[0])
print('数据条数：', df_incubator_mxnet_pr_data.shape[0])
print('数据条数：', df_issue_content_mx.shape[0])
print('数据条数：', df_pr_content_mx.shape[0])

df_issue_content_mx['IssueComments'] = df_issue_content_mx.IssueComments.apply(load_json)
df_pr_content_mx['PrComments'] = df_pr_content_mx.PrComments.apply(load_json)
df_incubator_mxnet_pr_data['IsMPLF'] = df_incubator_mxnet_pr_data.Files.apply(dulge_ifMpl)

# 创建 sentiment_analysis 列表
sentiment_analysis_mx = []

# 调用函数以转换数据并填充 new_rows
convert_issueData_to_sentimentAnalysis(df_issue_content_mx, df_incubator_mxnet_pr_data, df_incubator_mxnet_issue_data, sentiment_analysis_mx)
convert_prData_to_sentimentAnalysis(df_pr_content_mx, df_incubator_mxnet_pr_data, df_incubator_mxnet_issue_data, sentiment_analysis_mx)
# 将收集的行列表转换为 DataFrame
df_sentiment_analysis_mx = pd.DataFrame(sentiment_analysis_mx)
print(df_sentiment_analysis_mx.shape[0])

df_sentiment_analysis_mx['CleanedTextContent'] = df_sentiment_analysis_mx.TextContent.apply(preprocess_text)
df_sentiment_analysis_mx.to_csv('./processing_sentiment_analysis_mxComments_v1.0.csv', index=False)

C:\Users\shone\AppData\Local\Temp\ipykernel_20208\4263502064.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


数据条数： 190
数据条数： 259
数据条数： 190
数据条数： 244
1113


In [12]:
conn = connect_to_mysql('deep_learning')
df_pytorch_issue_data = fetch_data_from_mysql(conn, 'pytorch_issue_data', 'IssueID', 'RelatedPrId', 'Category')
df_pytorch_pr_data = fetch_data_from_mysql(conn, 'pytorch_pr_data', 'PrID', 'PrRelatedIssueId', 'PrFiles')
df_issue_content_py = fetch_data_from_mysql(conn, 'issue_content_py', 'IssueID', 'IssueContent', 'url', 'IssueCreator', 'IssueCreatedAt', 'IssueCommentsNum', 'IssueComments')
df_pr_content_py = fetch_data_from_mysql(conn, 'pr_content_py', 'PrID', 'PrContent', 'url', 'PrCreator', 'PrCreatedAt', 'PrCommentsNum', 'PrComments')
conn.close()

df_pytorch_issue_data.drop_duplicates(inplace=True)
df_pytorch_pr_data.drop_duplicates(inplace=True)
df_issue_content_py.drop_duplicates(inplace=True)
df_pr_content_py.drop_duplicates(inplace=True)

print('数据条数：', df_pytorch_issue_data.shape[0])
print('数据条数：', df_pytorch_pr_data.shape[0])
print('数据条数：', df_issue_content_py.shape[0])
print('数据条数：', df_pr_content_py.shape[0])

df_issue_content_py['IssueComments'] = df_issue_content_py.IssueComments.apply(load_json)
df_pr_content_py['PrComments'] = df_pr_content_py.PrComments.apply(load_json)
df_pytorch_pr_data['IsMPLF'] = df_pytorch_pr_data.PrFiles.apply(dulge_ifMpl)

# 创建 sentiment_analysis 列表
sentiment_analysis_py = []

# 调用函数以转换数据并填充 new_rows
convert_issueData_to_sentimentAnalysis(df_issue_content_py, df_pytorch_pr_data, df_pytorch_issue_data, sentiment_analysis_py)
convert_prData_to_sentimentAnalysis(df_pr_content_py, df_pytorch_pr_data, df_pytorch_issue_data, sentiment_analysis_py)
# 将收集的行列表转换为 DataFrame
df_sentiment_analysis_py = pd.DataFrame(sentiment_analysis_py)
print(df_sentiment_analysis_py.shape[0])

df_sentiment_analysis_py['CleanedTextContent'] = df_sentiment_analysis_py.TextContent.apply(preprocess_text)
df_sentiment_analysis_py.to_csv('./processing_sentiment_analysis_pyComments_v1.0.csv', index=False)

C:\Users\shone\AppData\Local\Temp\ipykernel_20208\4263502064.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


数据条数： 927
数据条数： 1068
数据条数： 927
数据条数： 1068
7166


In [13]:
conn = connect_to_mysql('deep_learning')
df_tensorflow_issue_data = fetch_data_from_mysql(conn, 'tensorflow_issue_data', 'IssueID', 'RelatedPrId', 'Category')
df_tensorflow_pr_data = fetch_data_from_mysql(conn, 'tensorflow_pr_data', 'PrID', 'PrRelatedIssueId', 'PrFiles')
df_issue_content_tf = fetch_data_from_mysql(conn, 'issue_content_tf', 'IssueID', 'IssueContent', 'url', 'IssueCreator', 'IssueCreatedAt', 'IssueCommentsNum', 'IssueComments')
df_pr_content_tf = fetch_data_from_mysql(conn, 'pr_content_tf', 'PrID', 'PrContent', 'url', 'PrCreator', 'PrCreatedAt', 'PrCommentsNum', 'PrComments')
conn.close()

df_tensorflow_issue_data.drop_duplicates(inplace=True)
df_tensorflow_pr_data.drop_duplicates(inplace=True)
df_issue_content_tf.drop_duplicates(inplace=True)
df_pr_content_tf.drop_duplicates(inplace=True)

print('数据条数：', df_tensorflow_issue_data.shape[0])
print('数据条数：', df_tensorflow_pr_data.shape[0])
print('数据条数：', df_issue_content_tf.shape[0])
print('数据条数：', df_pr_content_tf.shape[0])

df_issue_content_tf['IssueComments'] = df_issue_content_tf.IssueComments.apply(load_json)
df_pr_content_tf['PrComments'] = df_pr_content_tf.PrComments.apply(load_json)

df_tensorflow_pr_data['IsMPLF'] = df_tensorflow_pr_data.PrFiles.apply(dulge_ifMpl)

# 创建 sentiment_analysis 列表 调用函数以转换数据并填充 new_rows
sentiment_analysis_tf = []
convert_issueData_to_sentimentAnalysis(df_issue_content_tf, df_tensorflow_pr_data, df_tensorflow_issue_data, sentiment_analysis_tf)
convert_prData_to_sentimentAnalysis(df_pr_content_tf, df_tensorflow_pr_data, df_tensorflow_issue_data, sentiment_analysis_tf)

# 将收集的行列表转换为 DataFrame
df_sentiment_analysis_tf = pd.DataFrame(sentiment_analysis_tf)
print(df_sentiment_analysis_tf.shape[0])

df_sentiment_analysis_tf['CleanedTextContent'] = df_sentiment_analysis_tf.TextContent.apply(preprocess_text)
df_sentiment_analysis_tf.to_csv('./processing_sentiment_analysis_tfComments_v1.0.csv', index=False)

C:\Users\shone\AppData\Local\Temp\ipykernel_20208\4263502064.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


数据条数： 381
数据条数： 443
数据条数： 381
数据条数： 432
2348
